# StructuredOutputParser
- LLM 출력에서 여러 필드를 반환하고 싶을 때 사용. Pydantic/JSON Parser가 더 강력하지만, 그렇지 않은 모델에서 유용하다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


In [2]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    # max_new_tokens=1024,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)
model = ChatHuggingFace(llm=llm)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatHuggingFace` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface.chat_models import huggingface import ChatHuggingFace`.
  warn_deprecated(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="answer", description="Answer for user's question"),
    ResponseSchema(name="source", description="Referenced source to answer the user's question, it must be website url"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
prompt = PromptTemplate(
    template="Answer the user's question as best as possible\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // Answer for user\'s question\n\t"source": string  // Referenced source to answer the user\'s question, it must be website url\n}\n```'}, template="Answer the user's question as best as possible\n{format_instructions}\n{question}")

In [6]:
chain = prompt | model | output_parser
chain.invoke({"question": "What is capital city of South Korea?"})

{'answer': 'Seoul', 'source': 'https://en.wikipedia.org/wiki/South_Korea'}

In [10]:
for chunk in chain.stream({"question": "How many people live in South Korea?"}):
    print(chunk)

{'answer': 'approximately 51.8 million people', 'source': 'https://www.worldometers.info/world-population/korea/'}
